#### CIE4604 Simulation and Visualization
# Module 2 Satellite Orbits - Exercise 2

**Hans van der Marel, 18 November 2020**

Radarsat-2 should be a 24 day sun-synchronous repeat orbit. In this exercise we are going to check if this is indeed the case. And if not, we will look for an explanation.

## Import crsutil.py and tleplot.py Python modules

For the first time, download `CIE4604-M2-python.zip` from Brightspace and unzip this in your current working directory. This should give you two Python modules: `crsutil.py` and `tleplot.py`.

For this Jupyter notebook to work, the two Python modules should be in the same folder as this notebook. Import the modules using the following statements:

In [ ]:
import numpy as np
import crsutil as crs
import tleplot as tle

To get the docs for a specific function, use `help(functionname)`, or, to get the docs on all the functions at once do `help(module)`. To get help in a separate window type `module?` or `functionname?`. 

## Compute satellite position and velocity for Radarsat-2

First read the Two Line Elements (TLE) for Earth Resource Satellite, that contain Radarsat-2 (amongst others), and save the 
results in a structure array *tleERS* with `tleread` 
```
tleERS = tleread('resource-10-oct-2017.tle', verbose=0)
```
To suppress output during the reading we have added the option value pair `verbose=0` to the call.

To check that Radarsat-2 is in a 24 day sun-synchronous repeat orbit we compute two satellite positions 24 days apart. 

I have selected an epoch for which i know Radarsat-2 is in the proximity of Delft. At this time Radarsat-2 is on a descending track East of Delft. However, any other epoch would also be fine. The second epoch is 24 days later.
```
t[0] = tledatenum('2017-9-28 6:01')
t[1] = t[0] + 24
```
Then compute the satellite positions and velocities for both dates
```
datestr(t)
xsat, vsat = tle2vec(tleERS, t, 'RADARSAT-2')
```

In [ ]:
tleERS = tle.tleread('resource-10-oct-2017.tle', verbose=0)

t0 = tle.tledatenum('2017-9-28 6:01')
#t = np.array([ t0 , t0 + 24 ])
t = np.hstack([ t0 , t0 + 24 ])
print('t',t)

xsat, vsat = tle.tle2vec(tleERS, t, 'RADARSAT-2')

print('xsat: ', xsat)
print('vsat: ', vsat)


In [ ]:
tleERS = tle.tleread('resource-10-oct-2017.tle', verbose=0)

t = tle.tledatenum(['2017-9-28 6:01',24*24*60,24*24*60])
print('t',t)

xsat, vsat = tle.tle2vec(tleERS, t, 'RADARSAT-2')
print('xsat: ', xsat)
print('vsat: ', vsat)

## Compute the difference between the two positions

If the repeat period is indeed 24 days the two positions should be almost identical. Check this by computing the difference between both positions.

Since we will be comparing positions more than once, we will first write a function to compute the difference and print the differences. We call this function `printPosDiff`:

In [ ]:
def printPosDiff(xsat,vsat, ECEF=False):
    # Compute Difference between the two positions and print
    
    delta_pos=xsat[1,:]-xsat[0,:]
    delta_dist = np.sqrt(np.sum(delta_pos**2))

    rsat = np.sqrt(np.sum(xsat**2,axis=1))
    velsat = np.sqrt(np.sum(vsat**2,axis=1))
    delta_along_track = np.dot(delta_pos,vsat[0,:]/velsat[1])

    delta_ra=( np.arctan2(xsat[1,1], xsat[1,0]) - np.arctan2(xsat[0,1], xsat[0,0]) ) * 180 / np.pi
    delta_dec=( np.arcsin(xsat[1,2]/rsat[1]) - np.arcsin(xsat[0,2]/rsat[0])) * 180 / np.pi

    if ECEF:
        print('Coordinate Difference ECEF [km]: {:>7.1f} {:>7.1f} {:>7.1f}\n'.format(delta_pos[0]/1000,delta_pos[1]/1000,delta_pos[2]/1000))
    else:
        print('Coordinate Difference ECI [km]:  {:>7.1f} {:>7.1f} {:>7.1f}\n'.format(delta_pos[0]/1000,delta_pos[1]/1000,delta_pos[2]/1000))
    print('Distance difference [km]:        {:>7.1f}'.format(delta_dist/1000))
    print('- Along track distance [km]      {:>7.1f}'.format(delta_along_track/1000))
    print('- Perpendicular distance [km]    {:>7.1f}'.format(np.sqrt(delta_dist**2-delta_along_track**2)/1000))
    print('- Radial distance [km]           {:>7.1f}\n'.format((rsat[1]-rsat[0])/1000))
    if ECEF:
       print('Longitude difference [deg]       {:>7.1f}'.format(delta_ra))
       print('Latitude difference [deg]        {:>7.1f}\n'.format(delta_dec))
    else:
       print('Right ascension difference [deg] {:>7.1f}'.format(delta_ra))
       print('Declination difference [deg]     {:>7.1f}\n'.format(delta_dec))
    

The `printPosDiff` function has also a few print statements to show the differences in the radial, along and across track directions; and to show the differences in right ascension and declination.

### Comparison in ECI

Now, let try this function on our *xsat* and *vsat* parameters...

In [ ]:
printPosDiff(xsat,vsat)

The two positions are some *2000* km apart, and your first impression may be, that this is not what we would expect for a repeat orbit orbit... But then, these are difference in an ECI reference frame, and for repeat orbits to be meaningful we have to be in an ECEF reference frame.

The main difference is in the right ascension, *23.6* degrees, which perfectly matches the rotation of the Earth around the sun for a 24 day period.

The next step is to convert the positions into an ECEF and then compare again.

### Comparison in ECEF

To convert the positions from ECI to ECEF we use the `eci2ecef` funtion from the `crsutil` module,
```
xsate, vsate = eci2ecef(t, xsat, vsat)
```
This functions needs the time t (why?).

In [ ]:
xsate, vsate = crs.eci2ecef(t, xsat, vsat)

print('xsate',xsate)
print('vsate',vsate)

The difference between the two postions in the ECEF frame now is

In [ ]:
printPosDiff(xsate, vsate, ECEF=True)

**This seems to have worked!!**

The difference in ECEF coordinates is less than *20* km, and mainly
in the along track direction. With a velocity of *7.5* km/s (check this
yourself `np.sqrt(np.sum(vsate**2,axis=1))` this corresponds to a *2*-*3* second
time difference. The differences in latitude and longitude are smaller
than a few tenths of a degree. 

The remaining differences are the result of small errors in the Two
Line Elements and limited accuracy of the orbit propagation method that
is used in our Python routines. 

Also most satellites have to do small orbit adjustments from time to time to stay within the envellope of
their design orbit. This is definitely not taken into account in the orbit propagation!

## Defunct orbit propagation

The default orbit propagation method includes the effects of **J2** on
the secular component of the orbit elements. We can turn this off
in the computations by providing the `propagation='NOJ2'` option to `tle2vec`.

In [ ]:
xsatNoJ2, vsatNoJ2 = tle.tle2vec(tleERS,t,'RADARSAT-2',propagation='NOJ2')

The difference between the two postions in the ECEF frame is

In [ ]:
xsateNoJ2, vsateNoJ2 = crs.eci2ecef(t, xsatNoJ2, vsatNoJ2)
printPosDiff(xsateNoJ2, vsateNoJ2, ECEF=True)

This is the result of improper orbit propagation. As you can observe the effect is quite dramatic.

We need to properly propagate the effect of J2 on the orbital elements.
For more information how this is achieved see the lecture notes and
the Matlab function `tle2orb`.

[End of this Jupyter notebook]